# Fletcher Project: Summarizing Tweets

In [32]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.corpus import treebank_chunk
from textblob import TextBlob
import pymongo
from nltk.corpus import stopwords
from nltk.corpus import movie_reviews
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict
from operator import itemgetter
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from __future__ import division, unicode_literals
import operator
import requests
from requests_oauthlib import OAuth1
from pymongo import MongoClient
import tweepy
import cnfg
import numpy as np
import re

## Opening Twitter API

In [33]:
config = cnfg.load(".twitter_config")

In [34]:
auth = tweepy.OAuthHandler(config['consumer_key'],
              config['consumer_secret'])
auth.set_access_token(config['access_token'],
              config['access_token_secret'])

In [35]:
api = tweepy.API(auth)

## Choosing Twitter Account

In [36]:
account = 'YahooFinance'

## Getting Tweeter Data into a List

In [37]:
data_tweets = api.user_timeline(account, count=200)
tweets = []
tweets_text = []
for tweet in data_tweets:
    tweets.append(tweet.text)
len(tweets)

200

In [38]:
tweets

[u'Rate hikes and market returns http://t.co/gYRo694xFJ via @WildcatTrader $SPY',
 u'Where is your compass pointing today? http://t.co/UoZgP1EltI via @jaltucher http://t.co/GS3dxljPLC',
 u'June\u2019s first trading day hit hard in four of last seven years http://t.co/MWuY4yavS1 via @AlmanacTrader $SPY',
 u'Can Bob Evans escape activist Sandell by monetizing real estate? http://t.co/1wyejfZqAu via @TheDealNewsroom $BOBE http://t.co/g7QGdkxkXy',
 u"Carl's, Hardee's CEO: Not all models know how to eat burgers http://t.co/qLiuJIvlIE",
 u'RT @mandiwoodruff: Staggering: you\u2019d have to work 98 hours per week to afford a one-bedroom in NYC: http://t.co/PYNpNpKm86 http://t.co/M2oN\u2026',
 u'Altera, Intel up on deal talk; GameStop unstoppable; Splunk flunks http://t.co/3t7BIt6twr',
 u'Watch live at noon: Nike and Under Armour face off, Investors hoard cash and tech M&amp;A heats up http://t.co/iDd6TAGVhT',
 u'How to invest in the Cybersecurity boom http://t.co/LH7R7SOjx5 via @JeffReevesIP $

## Storing Tweets into Mongo DB

In [39]:
client = MongoClient()

In [40]:
datatweets = client.dsbc.datatweets

In [41]:
datatweets.remove( { } )

{u'n': 200, u'ok': 1}

In [42]:
datatweets = client.dsbc.datatweets
for k in tweets:
    datatweets_d = {'text': k}
    datatweets.insert(datatweets_d)

In [43]:
tweets = []
cursor = datatweets.find({})
for d in range(datatweets.count()):
    tweet_text = cursor.next()
    print tweet_text
    tweets.append(tweet_text['text'])
tweets

{u'text': u'Rate hikes and market returns http://t.co/gYRo694xFJ via @WildcatTrader $SPY', u'_id': ObjectId('5568a805bb9ab40372a2f49b')}
{u'text': u'The Next Obamacare Fight: Who to Blame http://t.co/FifN8DIHrS', u'_id': ObjectId('5568a805bb9ab40372a2f4a9')}
{u'text': u'US presses Europeans at meeting to solve Greek crisis http://t.co/sZ2UhuQA9f', u'_id': ObjectId('5568a805bb9ab40372a2f4ad')}
{u'text': u'American households feeling less financially stressed http://t.co/epxHTLSu9W', u'_id': ObjectId('5568a805bb9ab40372a2f4af')}
{u'text': u'US economy shrank at 0.7 percent rate in first quarter http://t.co/30PGYlca47', u'_id': ObjectId('5568a805bb9ab40372a2f4b0')}
{u'text': u'Early movers: $BIG, $GME, $DECK, $AMZN, $EQIX &amp; more http://t.co/XSwjrro86k', u'_id': ObjectId('5568a805bb9ab40372a2f4b1')}
{u'text': u'Amazon Plans to Add Its Own Line of Food http://t.co/VhWQTX63BQ', u'_id': ObjectId('5568a805bb9ab40372a2f4b2')}
{u'text': u'The Chinese Stock Rally Keeps Minting More Billionair

[u'Rate hikes and market returns http://t.co/gYRo694xFJ via @WildcatTrader $SPY',
 u'The Next Obamacare Fight: Who to Blame http://t.co/FifN8DIHrS',
 u'US presses Europeans at meeting to solve Greek crisis http://t.co/sZ2UhuQA9f',
 u'American households feeling less financially stressed http://t.co/epxHTLSu9W',
 u'US economy shrank at 0.7 percent rate in first quarter http://t.co/30PGYlca47',
 u'Early movers: $BIG, $GME, $DECK, $AMZN, $EQIX &amp; more http://t.co/XSwjrro86k',
 u'Amazon Plans to Add Its Own Line of Food http://t.co/VhWQTX63BQ',
 u'The Chinese Stock Rally Keeps Minting More Billionaires http://t.co/5RyDIMiA5l',
 u'The top 10 highest-paid female CEOs http://t.co/ZUfix8448I',
 u"Everyone Is Fleeing Oil's Biggest Fund http://t.co/NSWV9V8JzO",
 u'Oil gains 1 percent on U.S. inventory drop http://t.co/6YAy62by5K',
 u"Paul Singer: This is the new 'big short.' http://t.co/lanzYj91o3",
 u'.@jimcramer: The single hottest stock I follow. http://t.co/sRwKwZW0Lf',
 u'Google takes on

## Summarizing all most recent 200 Tweets

In [44]:
stopw = [str('http'), str('https'), str('['), str(']'), str('rt'), str('%'), str('@'), str('#')]
stopngram = stopwords.words('english')
stopngram += ['.', ',', '(', ')', "'", '"', 'http', 'RT']

In [45]:
adjectives = {}
nouns = {}
adverbs = {}
verbs = {}
for t in tweets:
    words = word_tokenize(t)
    for p in pos_tag(words):
        tags = p[1][:2]
        keys = p[0].encode('utf8')
        keys = keys.lower()   
        if keys not in stopw and str('/') not in keys and "\\" not in r"%r" % keys:
            if tags == 'NN':
                nouns.setdefault(keys,0)
                nouns[keys] += 1

## Most 10 used Nouns, Adverbs in all 200 recent Tweets

In [46]:
for words, count in sorted(nouns.items(), key = itemgetter(1), reverse=True)[:10]:
    print '%20s %i' % (words, count)

                 spy 17
                kors 14
              market 13
                u.s. 13
              stocks 11
          bidnessetc 9
               stock 9
                 ceo 9
               rates 7
             michael 6


## Most 10 used 2-grams, 3-grams, 4-grams in all 200 recent Tweets

In [47]:
counter = defaultdict(int)
n = 2
for doc in tweets:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:10]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

      via Bidnessetc 9
        Michael Kors 6
     via awealthofcs 4
    via CFAinstitute 4
        via Estimize 4
 via QuantifiedAlpha 3
      interest rates 3
     via peterckenny 3
 via TheDealNewsroom 3
     awealthofcs SPY 3


In [48]:
counter = defaultdict(int)
n = 3
for doc in tweets:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:10]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

via AlmanacTrader SPY 3
 via awealthofcs SPY 3
better faster cheaper 2
  real reason behind 2
don’t strangle nascent 2
Circuit poised outperformance 2
   getting work done 2
    could use wakeup 2
 U.S airlines stocks 2
Jabil Circuit poised 2


In [49]:
counter = defaultdict(int)
n = 4
for doc in tweets:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:10]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

stock market willing lose 2
interest rates It 's 2
  The man sold world 2
       Z 's Tidal Is 2
Why Oracle solid castle 2
Fed Please don’t strangle 2
   Study shows 11 US 2
Salesforce makes sense doesn’t 2
sold world fiat money 2
Elderly Are Staging Heists 2


## Selecting a Keyword

In [72]:
keyword = 'stock'

In [73]:
keyword = str(keyword)
keyword = keyword.lower()
keyword

'stock'

In [74]:
tweets_match = []
for r in tweets:
    if keyword in r.lower():
        tweets_match.append(r)

len(tweets_match)

21

In [75]:
adjectives = {}
nouns = {}
adverbs = {}
verbs = {}
for t in tweets_match:
    words = word_tokenize(t)
    for p in pos_tag(words):
        tags = p[1][:2]
        keys = p[0].encode('utf8')
        keys = keys.lower()   
        if keys not in stopw and str('/') not in keys and keyword not in keys and "\\" not in r"%r" % keys:
            if tags == 'JJ':
                adjectives.setdefault(keys,0)
                adjectives[keys] += 1
            if tags == 'NN':
                nouns.setdefault(keys,0)
                nouns[keys] += 1
            if tags == 'RB':
                adverbs.setdefault(keys,0)
                adverbs[keys] += 1
            if tags == 'VB':
                verbs.setdefault(keys,0)
                verbs[keys] += 1

## Most 5 used Verbs, Adjectives, Nouns, Adverbs in Tweets with Keyword

In [76]:
for words, count in sorted(verbs.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

                  is 4
              rising 3
                  be 3
                 win 2
                have 2


In [77]:
for words, count in sorted(adjectives.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

             willing 2
              little 1
          individual 1
             chinese 1
              dollar 1


In [78]:
for words, count in sorted(nouns.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

                 spy 6
                u.s. 4
              market 3
         awealthofcs 3
                year 2


In [79]:
for words, count in sorted(adverbs.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

                 fly 2
               again 2
                here 2
                 not 2
              mostly 1


## Most 5 used 2-grams, 3-grams, 4-grams in Tweets with Keyword

In [80]:
counter = defaultdict(int)
n = 2
for doc in tweets_match:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:5]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

     via awealthofcs 3
           win stock 2
        U.S airlines 2
      Bidnessetc UAL 2
      via Bidnessetc 2


In [81]:
counter = defaultdict(int)
n = 3
for doc in tweets_match:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:5]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

        To win stock 2
  via Bidnessetc UAL 2
stock market willing 2
airlines stocks Here 2
 market willing lose 2


In [82]:
counter = defaultdict(int)
n = 4
for doc in tweets_match:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:5]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

stock market willing lose 2
U.S airlines stocks Here 2
stocks Here might fly 2
 To win stock market 2
airlines stocks Here might 2
